<a href="https://colab.research.google.com/github/brighthann/The-Market-Gap-Analysis/blob/main/The%20%22Sugar%20Trap%22%20Market%20Gap%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#installing streamlit & plotly for a more interactive dashboard
!pip install streamlit plotly -q

#import libraries
import pandas as pd
import numpy as np
import re
from collections import Counter
import os

raw_data_dir = 'data/raw'
processed_data_dir = 'data/processed'
output_dir = 'outputs'

for folder in [raw_data_dir, processed_data_dir, output_dir]:
    os.makedirs(folder, exist_ok=True)

#dowloading dataset and storing source filename
Data_src = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"

raw_data_file = os.path.join(raw_data_dir, 'en.openfoodfacts.org.products.csv.gz')

#remove corrupted file if it exists
if os.path.exists(raw_data_file):
  print(f"Removing corrupted file: {raw_data_file}")
  os.remove(raw_data_file)
  print("File removed")

for f in os.listdir('.'):
  if f.endswith('.gz') and os.path.isfile(f):
    print(f"Removing stray file: {f}")
    os.remove(f)

#check if file already exist to avoid wasting memory after easch disconnect
if not os.path.exists(raw_data_file):
  !wget -O {raw_data_file} --progress=bar:force "{Data_src}" 2>&1
  print("\nDownloaded")
else:
  print(f"{raw_data_file} already exists")



Removing corrupted file: data/raw/en.openfoodfacts.org.products.csv.gz
File removed
Removing stray file: en.openfoodfacts.org.products.csv.gz
--2026-02-16 02:41:42--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 213.36.253.214
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|213.36.253.214|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://openfoodfacts-ds.s3.eu-west-3.amazonaws.com/en.openfoodfacts.org.products.csv.gz [following]
--2026-02-16 02:41:43--  https://openfoodfacts-ds.s3.eu-west-3.amazonaws.com/en.openfoodfacts.org.products.csv.gz
Resolving openfoodfacts-ds.s3.eu-west-3.amazonaws.com (openfoodfacts-ds.s3.eu-west-3.amazonaws.com)... 3.5.205.175, 3.5.205.118
Connecting to openfoodfacts-ds.s3.eu-west-3.amazonaws.com (openfoodfacts-ds.s3.eu-west-3.amazonaws.com)|3.5.205.175|:443... connected.
HTTP request sent, awaiting 

To work with a reasonable portion of the entire dataset. colab memory limit is ~12-13GB. First 500,000 rows should be loaded properly to avoid memory and performance issues. Only relevant columns with relation to roles will be loaded


In [11]:
#loading data sample
#columns that are relevant to roles
useful_columns = [
    #core nutrition
    'product_name', 'sugars_100g', 'proteins_100g', 'fat_100g', 'fiber_100g', 'energy_100g',
    #categories
    'categories_tags',
    #ingredients
    'ingredients_text',
    #extras
    'brands', 'countries_en',

]

rows_limit = 500000
df_raw = pd.read_csv(raw_data_file, sep='\t',nrows=rows_limit, usecols=useful_columns, low_memory=False, on_bad_lines='skip', encoding='utf-8')

print("\nData loading complete")
print(f"{df_raw.shape[0]:,} rows, {df_raw.shape[1]} colums")
print(f"\nColumn names: {df_raw.columns.tolist()}")
print(f"\nFirst 5 rows: ")
df_raw.head(5)




Data loading complete
500,000 rows, 10 colums

Column names: ['product_name', 'brands', 'categories_tags', 'countries_en', 'ingredients_text', 'energy_100g', 'fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g']

First 5 rows: 


,product_name,brands,categories_tags,countries_en,ingredients_text,energy_100g,fat_100g,sugars_100g,fiber_100g,proteins_100g
0,NaN,NaN,NaN,Germany,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Germany,NaN,NaN,NaN,NaN,NaN,NaN


Strategy Director: Data Ingestion & "The Clean Up".
 The goal is to produce a clean dataset where rows contain valid nutritional information. Critical columns are the ones essential for analysis and valid results. Products without names, sugar, and protein values cannot be used. Rows in which any of the useful/critical colums are null will be dropped.

In [12]:
df = df_raw.copy()

critical_col = ['product_name', 'sugars_100g', 'proteins_100g']
rows_before = len(df)
df.dropna(subset=critical_col, inplace=True)
rows_aft_null_drop = len(df)
print(f"Dropped {rows_before - rows_aft_null_drop:,} rows")
print(f"Remaining {rows_aft_null_drop:,} rows")

#nutrient colums convert to float
nutrient_col = ['sugars_100g', 'proteins_100g','fat_100g', 'fiber_100g', 'energy_100g']

for col in nutrient_col:

  if col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df.dropna(subset=['sugars_100g', 'proteins_100g'], inplace=True)

rows_aft_coerce = len(df)
print(f"\nColums converted to numeric. non-numeric artifacts dropped")
print(f"Dropped {rows_aft_null_drop - rows_aft_coerce:,} rows")
print(f"Remaining {rows_aft_coerce:,} rows")

#handling biologically impossible val
rows_before_outlier = len(df)

valid_mask = (
    (df['sugars_100g'] >= 0) & (df['sugars_100g'] <= 100) &
    (df['proteins_100g'] >= 0) & (df['proteins_100g'] <= 100)
)

df = df[valid_mask]

rows_aft_outlier = len(df)
print(f"\nRemoved biologically impossible values")
print(f"Dropped {rows_before_outlier - rows_aft_outlier:,} rows")
print(f"Remaining {rows_aft_outlier:,} rows")

#remove duplicates
rows_before_dup = len(df)
df.drop_duplicates(subset=['product_name', 'sugars_100g', 'proteins_100g'], keep='first', inplace=True)
rows_aft_dup = len(df)
print(f"\nRemoved duplicates")
print(f"Dropped {rows_before_dup - rows_aft_dup:,} rows")
print(f"Remaining {rows_aft_dup:,} rows")

#reset index to make up fot dropped rows
df.reset_index(drop=True, inplace=True)

print(f"\nClean Up complete")
print(f"original rows: {len(df_raw):,}")
print(f"rows removed: {len(df_raw) - len(df):,}")
print(f"clean rows: {len(df):,}")

cleaned_data_file = os.path.join(processed_data_dir, 'cleaned_data.csv')
df.to_csv(cleaned_data_file, index=False)


Dropped 98,924 rows
Remaining 401,076 rows

Colums converted to numeric. non-numeric artifacts dropped
Dropped 0 rows
Remaining 401,076 rows

Removed biologically impossible values
Dropped 5,979 rows
Remaining 395,097 rows

Removed duplicates
Dropped 46,116 rows
Remaining 348,981 rows

Clean Up complete
original rows: 500,000
rows removed: 151,019
clean rows: 348,981


Product Manager: Category Wrangler.
 Messy tags should be turned into clean readable high-level categories. a category map will be created containing category names and keywords belonging to those names.

In [13]:
category_map = [
    ("protein and health bars", ["protein-bar", "protein-snack", "energy-bar", "health-bar", "nutrition-bar", "granola-bar", "cereal-bar", "fruit-bar"]),
    ("chips and savory snacks", ["chips", "crisps", "popcorn", "pretzels", "savory-snack", "savoury-snack", "puffed", "crackers", "rice-cakes", "corn-snack"]),
    ("cookies and biscuits", ["cookie", "biscuit", "wafer", "shortbread"]),
    ("chocolates and candy", ["chocolate", "candy", "sweet-snack", "gummy", "candy-bar", "lollipop", "caramel", "toffee"]),
    ("nuts and seed", ["nuts", "almond", "cashew", "peanut", "walnut", "pistachio", "seed", "nut-butter"]),
    ("dried fruits and fruit snacks", ["dried-fruit", "fruit-snack", "raisin", "berry", "apricot", "prune"]),
    ("other snacks", ["snack"])
]

#function to take vals fromcategory tags
#take category tags string for product and return category label
def assign_prim_category(tags_string):

  if not isinstance(tags_string, str) or tags_string.strip() == "":
    return "uncategorized"

  tags_lower = tags_string.lower()
  for category_name, keywords in category_map:

    for keyword in keywords:

      if keyword in tags_lower:
        return category_name

  return "uncategorized"

#apply func to every element in column
print("\nCategories assigned")

df['primary_category'] = df['categories_tags'].apply(assign_prim_category)

#evaluation
category_counts = df['primary_category'].value_counts()
print(f"\n{category_counts.to_string()}")

uncatgrzd_frac = (df['primary_category'] == 'uncategorized').sum() / len(df) * 100
print(f"\nUncategorized products: {uncatgrzd_frac:.1f}%")

df_categorized = df[df['primary_category'] != 'uncategorized'].copy()
df_categorized.reset_index(drop=True, inplace=True)

print(f"\nFinal categorized dataset: {len(df_categorized):,} products across {df_categorized['primary_category'].nunique()} categories")

categorized_data_file = os.path.join(processed_data_dir, 'categorized_products.csv')
df_categorized.to_csv(categorized_data_file, index=False)



Categories assigned

primary_category
uncategorized                    290526
other snacks                      17612
chocolates and candy              15333
cookies and biscuits              13255
chips and savory snacks            5449
nuts and seed                      5077
dried fruits and fruit snacks      1008
protein and health bars             721

Uncategorized products: 83.2%

Final categorized dataset: 58,455 products across 7 categories


Marketing Lead: The "Nutrient Matrix" Visualization.


In [14]:
#calculate thresholds
protein_high_threshold = df_categorized['proteins_100g'].quantile(0.70)
sugar_low_threshold    = df_categorized['sugars_100g'].quantile(0.30)

print(f"High protein threshold: {protein_high_threshold:.1f}g per 100g")
print(f"Low Sugar threshold:    {sugar_low_threshold:.1f}g per 100g")

#high protein and low sugar
df_categorized['is_high_protein_low_sugar'] = ((df_categorized['proteins_100g'] >= protein_high_threshold) & (df_categorized['sugars_100g'] <= sugar_low_threshold))

high_protein_count = df_categorized['is_high_protein_low_sugar'].sum()
print(f"\nProducts in the 'High protein + Low sugar' quadrant: {high_protein_count:,}")

#count high-protein-low-sugar (hpls) prods by category
hpls_by_category = (df_categorized[df_categorized['is_high_protein_low_sugar']]['primary_category'].value_counts())
print("\nhigh protein + low sugar products by category:")
print(hpls_by_category.to_string())


High protein threshold: 7.1g per 100g
Low Sugar threshold:    6.0g per 100g

Products in the 'High protein + Low sugar' quadrant: 8,076

high protein + low sugar products by category:
primary_category
other snacks                     3444
nuts and seed                    2257
chips and savory snacks          1619
cookies and biscuits              594
chocolates and candy              113
protein and health bars            38
dried fruits and fruit snacks      11


Client: The Recommendation.

In [15]:
#total products per category
total_by_category = df_categorized['primary_category'].value_counts()

#ratio of healthy prods to total prods per category
health_ratio = (hpls_by_category / total_by_category).fillna(0)
health_ratio.name = 'health_ratio'

#summary table
category_summary = pd.DataFrame({'total_products': total_by_category, 'healthy_products': hpls_by_category, 'health_ratio': health_ratio}).fillna(0).sort_values('total_products', ascending=False)

print(f"\nCategory Health Summary: {category_summary.to_string()}")

#category with most prods but below avg health ratio - opportunity category
avg_health_ratio = health_ratio.mean()
opportunity_categories = category_summary[category_summary['health_ratio'] < avg_health_ratio].sort_values('total_products', ascending=False)

if len(opportunity_categories) > 0:
  top_opportunity = opportunity_categories.index[0]
else:
  top_opportunity = category_summary.index[0]

#nutritional profile for recomendation - average protein and sugar of prods in hpls
hpls_products = df_categorized[df_categorized['is_high_protein_low_sugar']]
recommended_protein = round(hpls_products['proteins_100g'].median(), 1)
recommended_sugar = round(hpls_products['sugars_100g'].median(), 1)

recommendation_text = (
    f"Based on the data, the biggest market opportunity is in {top_opportunity}, specifically targeting products with {recommended_protein}g of protein and less than {recommended_sugar}g of sugar"
)

print(f"\nRecommendation: {recommendation_text}")




Category Health Summary:                                total_products  healthy_products  health_ratio
primary_category                                                             
other snacks                            17612              3444      0.195548
chocolates and candy                    15333               113      0.007370
cookies and biscuits                    13255               594      0.044813
chips and savory snacks                  5449              1619      0.297119
nuts and seed                            5077              2257      0.444554
dried fruits and fruit snacks            1008                11      0.010913
protein and health bars                   721                38      0.052705

Recommendation: Based on the data, the biggest market opportunity is in chocolates and candy, specifically targeting products with 13.3g of protein and less than 3.2g of sugar


Consumer: The "Hidden Gem"

In [16]:
#get most common protein ingredients from high protein products
protein_sources = [
    'whey', 'casein', 'pea protein', 'soy protein', 'egg white',
    'egg', 'peanut', 'peanuts', 'almond', 'almonds', 'walnut',
    'cashew', 'sunflower seed', 'pumpkin seed', 'hemp seed',
    'quinoa', 'chickpea', 'lentil', 'chicken', 'beef', 'tuna',
    'salmon', 'milk protein', 'skimmed milk', 'skim milk',
    'dried whey', 'milk powder', 'soy', 'oat'
]

hp_ingredients = hpls_products['ingredients_text'].dropna()

#how many products does each protein source appear in
protein_source_count = Counter()

for ingredients_text in hp_ingredients:
  ingredients_lower = ingredients_text.lower()

  for source in protein_sources:

    if source in ingredients_lower:
      protein_source_count[source] += 1


top_protein_sources = protein_source_count.most_common(10)
print("\nTop 10 protein sources found in high-protein snack ingredients:")

for rank, (ingredient, count) in enumerate(top_protein_sources, 1):
  prcntge = count / len(hp_ingredients) * 100
  print(f"  {rank}. {ingredient.title():<20} appears in {count:,} products ({prcntge:.1f}%)")

#top 3 ingredients
top_three_ingredients = [src.title() for src, _ in top_protein_sources[:3]]
print(f"\nTop 3 protein sources for dashboard: {', '.join(top_three_ingredients)}")





Top 10 protein sources found in high-protein snack ingredients:
  1. Soy                  appears in 1,789 products (27.1%)
  2. Peanut               appears in 1,340 products (20.3%)
  3. Almond               appears in 964 products (14.6%)
  4. Almonds              appears in 939 products (14.2%)
  5. Peanuts              appears in 831 products (12.6%)
  6. Whey                 appears in 693 products (10.5%)
  7. Cashew               appears in 519 products (7.9%)
  8. Sunflower Seed       appears in 361 products (5.5%)
  9. Oat                  appears in 349 products (5.3%)
  10. Skim Milk            appears in 289 products (4.4%)

Top 3 protein sources for dashboard: Soy, Peanut, Almond


Creative Analyst: The "Candidate's Choice" Challenge.
After finding the 'Blue Ocean' in the snack aisle, the next thing that can be done is to find potential competitors. They can usually be considered as the market standard and it is important to know what they do, that makes them stand out from the rest. The brand with the most healthy products is what to look out for. health category for products will be determined by High protein, low sugar metrics.


In [17]:
#verify brands column exists
if 'brands' not in df_categorized.columns:
  raise ValueError("Brands column is missing")

#clean brands column
def clean_brand(raw_value):

#handle null values
  if not isinstance(raw_value, str) or raw_value.strip() == "":
    return "unknown"

#if multiple brands listed, take first one
  first_brand = raw_value.split(',')[0]

  return first_brand.strip().title()

#apply cleaning func to every row
df_categorized['primary_brand'] = df_categorized['brands'].apply(clean_brand)

#check how much brand data exists
total_with_brand = (df_categorized['primary_brand'] != 'unknown').sum()
total_without_brand = (df_categorized['primary_brand'] == 'unknown').sum()

print(f"\nProducts with a brand name: {total_with_brand:,}")
print(f"Products without a brand name: {total_without_brand:,}")

#aggregate by brand
#exclude unknown brands before grouping
df_known_brands = df_categorized[df_categorized['primary_brand'] != 'unknown']

brand_stats = df_known_brands.groupby('primary_brand').agg(
    total_products = ('product_name', 'count'),
    healthy_products = ('is_high_protein_low_sugar', 'sum'),
    avg_protein = ('proteins_100g', 'mean'),
    avg_sugar = ('sugars_100g', 'mean')
).reset_index()

#compute healthy percentage
brand_stats['healthy_frac'] = (brand_stats['healthy_products'] / brand_stats['total_products'] * 100).round(1)

#rounding up to 1 decimal place
brand_stats['avg_protein'] = brand_stats['avg_protein'].round(1)
brand_stats['avg_sugar'] = brand_stats['avg_sugar'].round(1)

#brands with at least 5 prods
min_products = 5

brand_leaderboard = brand_stats[brand_stats['total_products'] >= min_products].copy()

print(f"\nBrands with at least {min_products} products: {len(brand_leaderboard):,}")
print(f"Brands excluded: {len(brand_stats) - len(brand_leaderboard):,}")

#sort and rank
brand_leaderboard = brand_leaderboard.sort_values(by=['healthy_frac', 'total_products'], ascending=[False, False]).reset_index(drop=True)
brand_leaderboard['rank'] = range(1, len(brand_leaderboard) + 1)

#reorder columns
brand_leaderboard = brand_leaderboard[['rank', 'primary_brand', 'total_products', 'healthy_products', 'healthy_frac', 'avg_protein', 'avg_sugar']]

print("\nTop 5 brands by healthy product percentage")
print(brand_leaderboard.head(5).to_string(index=False))

#save
brand_file = os.path.join(output_dir, 'brand_leaderboard.csv')
brand_leaderboard.to_csv(brand_file, index=False)






Products with a brand name: 35,701
Products without a brand name: 22,754

Brands with at least 5 products: 1,131
Brands excluded: 4,136

Top 5 brands by healthy product percentage
 rank      primary_brand  total_products  healthy_products  healthy_frac  avg_protein  avg_sugar
    1    Sonoma Creamery              10                10         100.0         39.0        1.7
    2          Beer Nuts               8                 8         100.0         20.0        3.5
    3  Baked In Brooklyn               6                 6         100.0         12.8        2.9
    4       Fisher Snack               6                 6         100.0         18.5        3.6
    5 Hillshire Snacking               6                 6         100.0         19.7        0.6
